Cleveland fed derives these from model with current inflation and market data/forceasts, strips out risk premia. They have a cool inflation term structure that could be nice to show off instead


In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "cleveland_inflation"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [5]:
# The get_series_info method may be useful
fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "1-Year": "EXPINF1YR",
    "3-Year": "EXPINF3YR"
}

In [6]:
today = dt.date.today()

core_sticky_srs = fred.get_series(
    series_id=series_codes["1-Year"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m',
).rename("1-year expected inflation")

core_srs = fred.get_series(
    series_id=series_codes["3-Year"],
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
).rename("3-year expected inflation")


joined_df = pd.concat(
    [core_sticky_srs, core_srs], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

# Naming the index makes it easier tot title
joined_df.index.name = "Date"

joined_df.tail()

,1-year expected inflation,3-year expected inflation
Date,,
2025-05-01,2.691337,2.414413
2025-06-01,2.380542,2.358948
2025-07-01,2.789047,2.458601
2025-08-01,2.689360,2.387534
2025-09-01,2.807321,2.428930


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [7]:
# First make the figure
fig = go.Figure()

# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Cleveland Fed Estimated Inflation Expectations<br><sup>Monthly</sup>',),
    xaxis_title="Date",
    yaxis_title="Change (YoY)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%"
)

fig.add_hline(
    y=2,
    line_dash="dash",
    line_color="red",
    annotation_text="Longer-Run Target",
    annotation_position="right bottom"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

ut.add_end_labels(fig = fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")
